In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA

df = pd.read_csv('C:/Users/User/Desktop/10/data-2/Week2_challenge_data_source(CSV).csv')

print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

<span style="font-size: 36px;">Task 1.1</span>

In [35]:

# Identify duplicated 'Bearer Id' values
duplicate_bearer_ids = df['Bearer Id'].value_counts()[df['Bearer Id'].value_counts() > 1]

duplicates_sample = df[df['Bearer Id'].isin(duplicate_bearer_ids.index)].head(10)
print("Sample of records with duplicated 'Bearer Id':\n", duplicates_sample)

duration_analysis = df[df['Bearer Id'].isin(duplicate_bearer_ids.index)]
duration_stats = duration_analysis.groupby('Bearer Id')['Dur. (ms)'].describe()
print("\nDuration analysis for each duplicated 'Bearer Id':\n", duration_stats)

aggregated_data = duration_analysis.groupby('Bearer Id').agg({
    'Dur. (ms)': 'sum',
    'Total UL (Bytes)': 'sum',
    'Total DL (Bytes)': 'sum',
}).reset_index()

print("\nAggregated data for each duplicated 'Bearer Id':\n", aggregated_data.head())


Sample of records with duplicated 'Bearer Id':
        Bearer Id            Start  Start ms             End  End ms  \
20  1.304243e+19   4/17/2019 0:35     363.0  4/25/2019 4:52   691.0   
30  1.304243e+19   4/18/2019 0:35     711.0  4/25/2019 4:52   691.0   
43  1.304243e+19   4/19/2019 2:55     809.0  4/25/2019 0:17   262.0   
50  1.304243e+19   4/20/2019 2:55     691.0  4/25/2019 0:17   262.0   
58  7.277826e+18  4/20/2019 14:08     136.0  4/25/2019 0:54    79.0   
62  1.304243e+19   4/21/2019 2:55      73.0  4/25/2019 0:17   262.0   
63  1.304243e+19   4/21/2019 3:04     521.0  4/25/2019 4:02   116.0   
72  7.277826e+18  4/21/2019 13:57     724.0  4/25/2019 0:54    79.0   
82  1.304243e+19   4/22/2019 0:35     293.0  4/25/2019 0:35    30.0   
84  1.304243e+19   4/22/2019 1:17     301.0  4/25/2019 1:17   938.0   

    Dur. (ms)          IMSI  MSISDN/Number          IMEI Last Location Name  \
20   706649.0  2.082010e+14   3.365875e+10  8.654080e+13            T29714B   
30   620248.

<span style="font-size: 30px;">Number of xDR sessions</span>

In [36]:

xdr_sessions_per_user = df.groupby('IMSI')['Bearer Id'].count().reset_index()
xdr_sessions_per_user.columns = ['IMSI', 'Number of xDR Sessions']

print("Number of xDR sessions per user:\n", xdr_sessions_per_user.head())


Number of xDR sessions per user:
            IMSI  Number of xDR Sessions
0  2.040471e+14                       1
1  2.040808e+14                       1
2  2.082001e+14                       1
3  2.082001e+14                       1
4  2.082001e+14                       2


<span style="font-size: 30px;">Session duration</span>

In [38]:
aggregated_data = df.groupby('Bearer Id')['Dur. (ms)'].sum().reset_index()

aggregated_data.columns = ['Bearer Id', 'Total Session Duration (ms)']

print("Aggregated session duration per user:\n", aggregated_data.head())

Aggregated session duration per user:
       Bearer Id  Total Session Duration (ms)
0  6.917538e+18                      24534.0
1  6.917538e+18                      21489.0
2  6.917538e+18                      27786.0
3  6.917538e+18                      15635.0
4  6.917538e+18                      24264.0


<span style="font-size: 30px;">The total download (DL) and upload (UL) data</span>

In [39]:
aggregated_data = df.groupby('Bearer Id').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
}).reset_index()

aggregated_data.columns = ['Bearer Id', 'Total Download Data (Bytes)', 'Total Upload Data (Bytes)']

print("Aggregated total download and upload data per user:\n", aggregated_data.head())

Aggregated total download and upload data per user:
       Bearer Id  Total Download Data (Bytes)  Total Upload Data (Bytes)
0  6.917538e+18                  500721999.0                 41704610.0
1  6.917538e+18                  709549265.0                 38693596.0
2  6.917538e+18                  854465860.0                 15548926.0
3  6.917538e+18                  825786510.0                 49605688.0
4  6.917538e+18                  624957763.0                 21983463.0


<span style="font-size: 30px;">The total data volume (in Bytes) during this session for each application</span>

In [41]:
df['Total Data Volume (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

# Group by 'Bearer Id' to get the total data volume per user
aggregated_data = df.groupby('Bearer Id').agg({
    'Total Data Volume (Bytes)': 'sum',
 
}).reset_index()

print("Aggregated total data volume per user:\n", aggregated_data.head())

Aggregated total data volume per user:
       Bearer Id  Total Data Volume (Bytes)
0  6.917538e+18                542426609.0
1  6.917538e+18                748242861.0
2  6.917538e+18                870014786.0
3  6.917538e+18                875392198.0
4  6.917538e+18                646941226.0


<span style="font-size: 30px;">Task 1.2 </span>

In [2]:
print(df.info()) 
print(df.describe()) 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

<span style="font-size: 30px;">Handle Missing Values and Outliers</span>

In [3]:
df.fillna(df.mean(), inplace=True)

numeric_cols = df.select_dtypes(include=[np.number])
z_scores = np.abs(stats.zscore(numeric_cols))
df[numeric_cols.columns] = df[numeric_cols.columns].mask(z_scores > 3, df.mean(), axis=1)

print(df.isnull().sum())
print(df.describe())


C:\Users\User\AppData\Local\Temp\ipykernel_11056\330299129.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_11056\330299129.py:7: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[numeric_cols.columns] = df[numeric_cols.columns].mask(z_scores > 3, df.mean(), axis=1)


Bearer Id                                      0
Start                                          1
Start ms                                       0
End                                            1
End ms                                         0
Dur. (ms)                                      0
IMSI                                           0
MSISDN/Number                                  0
IMEI                                           0
Last Location Name                          1153
Avg RTT DL (ms)                                0
Avg RTT UL (ms)                                0
Avg Bearer TP DL (kbps)                        0
Avg Bearer TP UL (kbps)                        0
TCP DL Retrans. Vol (Bytes)                    0
TCP UL Retrans. Vol (Bytes)                    0
DL TP < 50 Kbps (%)                            0
50 Kbps < DL TP < 250 Kbps (%)                 0
250 Kbps < DL TP < 1 Mbps (%)                  0
DL TP > 1 Mbps (%)                             0
UL TP < 10 Kbps (%) 

<span style="font-size: 30px;">Variable Transformations – Segment Users</span>

In [4]:
df['Total Duration'] = df.groupby('Bearer Id')['Dur. (ms)'].transform('sum')

df['Decile Class'] = pd.qcut(df['Total Duration'], 5, labels=[1, 2, 3, 4, 5])

df['Total Data'] = df['Total UL (Bytes)'] + df['Total DL (Bytes)']
decile_data = df.groupby('Decile Class')['Total Data'].sum().reset_index()

print("\nTotal Data per Decile Class:")
print(decile_data)


Total Data per Decile Class:
  Decile Class    Total Data
0            1  1.487971e+13
1            2  1.777675e+13
2            3  1.195002e+13
3            4  1.536658e+13
4            5  1.439210e+13


<span style="font-size: 30px;">Analyze Basic Metrics</span>

In [5]:
mean_values = df.mean()
median_values = df.median()

print("\nMean Values:")
print(mean_values)
print("\nMedian Values:")
print(median_values)


C:\Users\User\AppData\Local\Temp\ipykernel_11056\2195426780.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values = df.mean()



Mean Values:
Bearer Id                                   1.013887e+19
Start ms                                    4.991882e+02
End ms                                      4.988009e+02
Dur. (ms)                                   9.870655e+04
IMSI                                        2.082017e+14
MSISDN/Number                               3.375412e+10
IMEI                                        4.847455e+13
Avg RTT DL (ms)                             8.525657e+01
Avg RTT UL (ms)                             1.380642e+01
Avg Bearer TP DL (kbps)                     1.131532e+04
Avg Bearer TP UL (kbps)                     1.151190e+03
TCP DL Retrans. Vol (Bytes)                 1.522457e+07
TCP UL Retrans. Vol (Bytes)                 5.816814e+05
DL TP < 50 Kbps (%)                         9.401156e+01
50 Kbps < DL TP < 250 Kbps (%)              2.450413e+00
250 Kbps < DL TP < 1 Mbps (%)               1.297854e+00
DL TP > 1 Mbps (%)                          1.039823e+00
UL TP < 10 Kbps (

C:\Users\User\AppData\Local\Temp\ipykernel_11056\2195426780.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  median_values = df.median()
